# Práctica 4. Procesamiento audio

##### Ejercicio 1: Construir un identificador de notas musicales. Es decir; en su versión más sencilla (y suficiente) la entrada es un sonido con una sola nota musical y debe identificar cuál es. Por simplicidad elija un único instrumento para la identificación.

- El caso más sencillo es el del piano, pero se valorará que se haga con otros instrumentos como la guitarra, la trompeta…
- Se valorará que se identifiquen octavas de notas

In [1]:
import numpy as np
from scipy.fft import fft, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt

    
def detector_frecuencias(audio_data, sample_rate):
    
    if len(audio_data.shape) == 2:
        audio_data = audio_data[:,0]
        
    audio_data = audio_data / np.max(np.abs(audio_data))    
    
    corr = np.correlate(audio_data, audio_data, mode="full")
    corr = corr[len(corr)//2:]
    
    d = np.diff(corr)
    start = np.where(d > 0)[0][0]
    
    peak = np.argmax(corr[start:]) + start
    frecuencia = sample_rate / peak
    
    return frecuencia   
     
    
def identificador_notas(frecuencia):
    notas = {
    'Do' : [16.3516],
    'Do#': [17.3239],
    'Re' : [18.3540],
    'Re#': [19.4454],
    'Mi' : [20.6017],
    'Fa' : [21.8268],
    'Fa#': [23.1246],
    'Sol' : [24.4997],
    'Sol#': [25.9565],
    'La' : [27.5000],
    'La#': [29.1353],
    'Si' : [30.8677]
    }
    
    notas_completas = {}
    for clave, valor in notas.items():
        lista = []
        for i in range(9):
            lista.append(valor[0] * (2**i))
        notas_completas[clave] = lista
    
    min_diff = float('inf')
    identificada = None
    octava = None

    for nota, frecuencias in notas_completas.items():
        for i, freq in enumerate(frecuencias):
            diff = abs(frecuencia - freq)
            if diff < min_diff:
                min_diff = diff
                identificada = nota
                octava = i

    return identificada, octava

In [3]:
sample_rate, audio_data = wavfile.read('Notas/Trompeta/trumpet_la.wav')

if len(audio_data.shape) == 2:
    audio_data = audio_data[:, 0]

frecuencia = detector_frecuencias(audio_data, sample_rate)
nota, octava = identificador_notas(frecuencia)

print(f"Frecuencia identificada: {frecuencia} Hz")
print(f"Octava {octava}, Nota: {nota}")

Frecuencia identificada: 436.3636363636364 Hz
Octava 4, Nota: La


###### APORTES ADICIONALES
- Identificación de acordes (complejo pero espectacular)
- Aportes adicionales

In [8]:
import numpy as np
from scipy.fft import fft, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

def identificador_acordes(audio_data, sample_rate, umbral=0.2):
    
    if len(audio_data.shape) == 2:
        audio_data = audio_data[:,0]
        
    audio_data = audio_data / np.max(np.abs(audio_data))    
    
    N = len(audio_data)
    T = 1/sample_rate
    yf = np.abs(fft(audio_data))[:N//2]
    xf = fftfreq(N, T)[:N//2]
    
    peaks, _ = find_peaks(yf, height=umbral*np.max(yf))
    
    notas = []
    for pico in peaks:
        freq = xf[pico]
        notas.append(freq)
    
    return list(set(notas))  

In [9]:
sample_rate, audio_data = wavfile.read('acorde.wav')

if len(audio_data.shape) == 2:
    audio_data = audio_data[:, 0]

frecuencia = identificador_acordes(audio_data, sample_rate)

count = 0
for picos in frecuencia:
    nota, octava = identificador_notas(picos)
    print(f"Frecuencia identificada: {picos} Hz")
    print(f"Octava {octava}, Nota: {nota}")
    count += 1

Frecuencia identificada: 130.78125 Hz
Octava 3, Nota: Do
Frecuencia identificada: 260.625 Hz
Octava 4, Nota: Do
Frecuencia identificada: 261.28125 Hz
Octava 4, Nota: Do
Frecuencia identificada: 261.0 Hz
Octava 4, Nota: Do
Frecuencia identificada: 554.4375 Hz
Octava 5, Nota: Do#
Frecuencia identificada: 554.8125 Hz
Octava 5, Nota: Do#
Frecuencia identificada: 370.03125 Hz
Octava 4, Nota: Fa#
Frecuencia identificada: 277.03125 Hz
Octava 4, Nota: Do#
Frecuencia identificada: 155.53125 Hz
Octava 3, Nota: Re#


C:\Users\Manuel\AppData\Local\Temp\ipykernel_14316\1522402451.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio_data = wavfile.read('acorde.wav')


##### Ejercicio 2: Construir una pequeña aplicación que permita operar con diferentes filtros (con un selector) y trabajar con varios umbrales. (uno para los filtros pasa-bajo y pasa-alto y dos para los filtros pasa-banda y rechaza-banda). Demuestre su funcionalidad con señales ruidosas. Muestre en cada filtrado la señal original y filtrada en el dominio temporal y en el dominio de la frecuencia.

###### APORTES ADICIONALES
- Reproducir la nota ruidosa original y filtrada a través de la aplicación para comparar resultados y cambiar parámetros en tiempo real (o casi).
- Aportes adicionales (como uso del micrófono, experimentar con implementaciones de filtros diferentes)